In [ ]:
# bewfore decode you get tokens 


encode-> add motion in between -> decode
encode the prev motion(bvh) -> 

In [ ]:
import torch
import numpy as np
from common.skeleton import Skeleton
from utils.paramUtil import *
from common.quaternion import qrot_np, qinv_np, qmul_np, quaternion_to_cont6d_np, qbetween_np, qfix

def joints_to_features(joints, feet_thre=0.002, dataset='humanml'):
    """
    joints: numpy array of shape (frames, joints_num, 3)
    returns: RIC feature vector (frames-1, feature_dim)
    """

    if dataset == 'humanml':
        n_raw_offsets = torch.from_numpy(t2m_raw_offsets)
        kinematic_chain = t2m_kinematic_chain
        joints_num = 22
        fid_r, fid_l = [8, 11], [7, 10]
        face_joint_indx = [2, 1, 17, 16]
    elif dataset == 'kit':
        n_raw_offsets = torch.from_numpy(kit_raw_offsets)
        kinematic_chain = kit_kinematic_chain
        joints_num = 21
        fid_r, fid_l = [14, 15], [19, 20]
        face_joint_indx = [11, 16, 5, 8]
    else:
        raise ValueError('Unknown dataset')

    joints = torch.from_numpy(joints).float()

    # Step 1: Build Skeleton
    skel = Skeleton(n_raw_offsets, kinematic_chain, 'cpu')

    # Step 2: Uniform skeleton (optional, skipped if already processed)
    tgt_offsets = skel.get_offsets_joints(joints[0])

    # Step 3: Put on floor
    floor_height = joints[:, :, 1].min()
    joints[:, :, 1] -= floor_height

    # Step 4: Face Z+
    r_hip, l_hip, sdr_r, sdr_l = face_joint_indx
    across1 = joints[0, r_hip] - joints[0, l_hip]
    across2 = joints[0, sdr_r] - joints[0, sdr_l]
    across = across1 + across2
    across = across / torch.norm(across)
    forward_init = torch.cross(torch.tensor([0.0, 1.0, 0.0]), across)
    forward_init = forward_init / torch.norm(forward_init)
    target = torch.tensor([0.0, 0.0, 1.0])
    root_quat_init = qbetween_np(forward_init.numpy(), target.numpy())
    root_quat_init = np.ones(joints.shape[:-1] + (4,)) * root_quat_init
    joints = qrot_np(root_quat_init, joints.numpy())

    global_positions = joints.copy()

    # Step 5: Foot contact detection
    def foot_detect(positions, thres):
        velfactor = np.array([thres, thres])
        feet_l_x = (positions[1:, fid_l, 0] - positions[:-1, fid_l, 0]) ** 2
        feet_l_y = (positions[1:, fid_l, 1] - positions[:-1, fid_l, 1]) ** 2
        feet_l_z = (positions[1:, fid_l, 2] - positions[:-1, fid_l, 2]) ** 2
        feet_l = ((feet_l_x + feet_l_y + feet_l_z) < velfactor).astype(np.float32)

        feet_r_x = (positions[1:, fid_r, 0] - positions[:-1, fid_r, 0]) ** 2
        feet_r_y = (positions[1:, fid_r, 1] - positions[:-1, fid_r, 1]) ** 2
        feet_r_z = (positions[1:, fid_r, 2] - positions[:-1, fid_r, 2]) ** 2
        feet_r = ((feet_r_x + feet_r_y + feet_r_z) < velfactor).astype(np.float32)

        return feet_l, feet_r

    feet_l, feet_r = foot_detect(joints, feet_thre)

    # Step 6: Extract features

    # Inverse kinematics to quaternions
    skel = Skeleton(n_raw_offsets, kinematic_chain, 'cpu')
    quat_params = skel.inverse_kinematics_np(joints, face_joint_indx, smooth_forward=True)
    quat_params = qfix(quat_params)

    # Continuous 6D
    cont_6d_params = quaternion_to_cont6d_np(quat_params)
    r_rot = quat_params[:, 0]

    # Root linear velocities
    velocity = joints[1:, 0] - joints[:-1, 0]
    velocity = qrot_np(r_rot[1:], velocity)
    l_velocity = velocity[:, [0, 2]]

    # Root rotation velocities
    r_velocity = qmul_np(r_rot[1:], qinv_np(r_rot[:-1]))
    r_velocity = np.arcsin(r_velocity[:, 2:3])

    # Root height
    root_y = joints[:-1, 0, 1:2]

    root_data = np.concatenate([r_velocity, l_velocity, root_y], axis=-1)

    # Joint positions (RIC)
    joints_local = joints.copy()
    joints_local[..., 0] -= joints_local[:, 0:1, 0]
    joints_local[..., 2] -= joints_local[:, 0:1, 2]
    joints_local = qrot_np(np.repeat(r_rot[:, None], joints_local.shape[1], axis=1), joints_local)
    ric_data = joints_local[:, 1:].reshape(joints_local.shape[0], -1)

    # Joint rotations
    rot_data = cont_6d_params[:, 1:].reshape(cont_6d_params.shape[0], -1)

    # Joint velocities
    local_vel = qrot_np(np.repeat(r_rot[:-1, None], global_positions.shape[1], axis=1),
                        global_positions[1:] - global_positions[:-1])
    local_vel = local_vel.reshape(local_vel.shape[0], -1)

    # Combine
    data = np.concatenate([root_data, ric_data[:-1], rot_data[:-1], local_vel, feet_l, feet_r], axis=-1)

    return data


In [3]:
test = np.load(r"generation\generation_name_nopredlen\joints\0\sample0_repeat0_len192_ik.npy", allow_pickle=True)  

In [4]:
test.shape

(468, 22, 3)

In [56]:
pred_motions_1.shape

(3, 200, 263)

In [5]:
ric_features = joints_to_features(test, feet_thre=0.002, dataset='humanml')

print(ric_features.shape)  

(467, 263)


C:\Users\TP\AppData\Local\Temp\ipykernel_20584\4195675775.py:46: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\Cross.cpp:66.)
  forward_init = torch.cross(torch.tensor([0.0, 1.0, 0.0]), across)


In [4]:
test.shape

(468, 22, 3)

In [12]:
mean = np.load(pjoin('checkpoints', opt.dataset_name, model_opt.vq_name, 'meta', 'mean.npy'))
std = np.load(pjoin('checkpoints', opt.dataset_name, model_opt.vq_name, 'meta', 'std.npy'))

In [13]:
normed_joints = (ric_features - mean) / std

In [71]:
normed_joints.shape

(467, 263)

In [14]:
import torch

ric_tensor = torch.from_numpy(normed_joints).float().unsqueeze(0).to('cpu')  # (1, seq_len, 263)


In [15]:
ric_tensor.shape

torch.Size([1, 467, 263])

In [16]:
quantized, tokens = vq_model.encode(ric_tensor)

In [61]:
pred_motions.shape

torch.Size([3, 200, 263])

In [62]:
ric_tensor.shape

torch.Size([1, 467, 263])

In [59]:
q.shape

torch.Size([3, 50, 6])

In [38]:
quantized.shape

torch.Size([1, 116, 6])

In [4]:
test_prompt=["A man is runnning"]
text_prompt="|".join(test_prompt)
generator.create_bvh_from_in(text_prompt)

Motion generation completed successfully! BVH files saved in ./generation\generation_name_nopredlen\animations


'./generation\\generation_name_nopredlen\\animations\\0\\sample0_repeat0_len80.bvh'

In [7]:
import sys
sys.argv = ['']  # Prevent argparse from picking up unwanted Jupyter arguments

# from gen_t2m import EvalT2MOptions  # or from wherever your file is

# Initialize and parse
parser = EvalT2MOptions()
opt = parser.parse(is_eval=True)

# Set values as per your CLI
opt.res_name = "tres_nlayer8_ld384_ff1024_rvq6ns_cdp0.2_sw"
opt.name = "2024-02-14-14-27-29_8_GPT_officialTrans_2iterPrdictEnd"
opt.text_prompt = "the person crouches and walks forward."
opt.motion_length = -1
opt.repeat_times = 3
opt.gpu_id = -1
opt.seed = 1
opt.ext = "generation_name_nopredlen"


------------ Options -------------
batch_size: 32
checkpoints_dir: ./log/t2m
cond_scale: 4
dataset_name: t2m
dropout: 0.2
ext: text2motion
ff_size: 1024
force_mask: False
gpu_id: -1
gumbel_sample: False
is_train: False
latent_dim: 384
mask_edit_section: None
max_motion_length: 196
motion_length: 0
n_heads: 6
n_layers: 8
name: t2m_nlayer8_nhead6_ld384_ff1024_cdp0.1_rvq6ns
num_batch: 2
repeat_times: 1
res_name: tres_nlayer8_ld384_ff1024_rvq6ns_cdp0.2_sw
seed: 1
source_motion: example_data/000612.npy
temperature: 1.0
text_path: 
text_prompt: 
time_steps: 10
topkr: 0.9
unit_length: 4
use_res_model: False
vq_name: rvq_nq1_dc512_nc512
which_epoch: latest
-------------- End ----------------


In [8]:
fixseed(opt.seed)

opt.device = torch.device("cpu" if opt.gpu_id == -1 else "cuda:" + str(opt.gpu_id))
torch.autograd.set_detect_anomaly(True)

dim_pose = 251 if opt.dataset_name == 'kit' else 263

# out_dir = pjoin(opt.check)
root_dir = pjoin(opt.checkpoints_dir, opt.dataset_name, opt.name)
model_dir = pjoin(root_dir, 'model')
result_dir = pjoin('./generation', opt.ext)
joints_dir = pjoin(result_dir, 'joints')
animation_dir = pjoin(result_dir, 'animations')
os.makedirs(joints_dir, exist_ok=True)
os.makedirs(animation_dir,exist_ok=True)

In [9]:
model_opt_path = pjoin(root_dir, 'opt.txt')
model_opt = get_opt(model_opt_path, device=opt.device)

Reading ./log/t2m\t2m\2024-02-14-14-27-29_8_GPT_officialTrans_2iterPrdictEnd\opt.txt


In [10]:
global vq_opt # This now works as vq_opt is global

vq_opt_path = pjoin('./log/vq', opt.dataset_name, model_opt.vq_name, 'opt.txt')
vq_opt = get_opt(vq_opt_path, device=opt.device)
vq_model, vq_opt = load_vq_model(vq_opt)

Reading ./log/vq\t2m\rvq_nq6_dc512_nc512_noshare_qdp0.2\opt.txt
Loading VQ Model rvq_nq6_dc512_nc512_noshare_qdp0.2 Completed!


c:\Users\TP\Desktop\genAI\BAMM\gen_t2m_c.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(pjoin(vq_opt.checkpoints_dir, vq_opt.dataset_name, vq_opt.nam

In [17]:
model_opt.num_tokens = vq_opt.nb_code
model_opt.num_quantizers = vq_opt.num_quantizers
model_opt.code_dim = vq_opt.code_dim

In [18]:
pjoin('checkpoints', opt.dataset_name, opt.res_name, 'opt.txt')

'checkpoints\\t2m\\tres_nlayer8_ld384_ff1024_rvq6ns_cdp0.2_sw\\opt.txt'

In [19]:
def load_res_model(res_opt,vq_opt):
    res_opt.num_quantizers = vq_opt.num_quantizers
    res_opt.num_tokens = vq_opt.nb_code
    res_transformer = ResidualTransformer(code_dim=vq_opt.code_dim,
                                            cond_mode='text',
                                            latent_dim=res_opt.latent_dim,
                                            ff_size=res_opt.ff_size,
                                            num_layers=res_opt.n_layers,
                                            num_heads=res_opt.n_heads,
                                            dropout=res_opt.dropout,
                                            clip_dim=512,
                                            shared_codebook=vq_opt.shared_codebook,
                                            cond_drop_prob=res_opt.cond_drop_prob,
                                            # codebook=vq_model.quantizer.codebooks[0] if opt.fix_token_emb else None,
                                            share_weight=res_opt.share_weight,
                                            clip_version=clip_version,
                                            opt=res_opt)

    ckpt = torch.load(pjoin(res_opt.checkpoints_dir, res_opt.dataset_name, res_opt.name, 'model', 'net_best_fid.tar'),
                      map_location=opt.device)
    missing_keys, unexpected_keys = res_transformer.load_state_dict(ckpt['res_transformer'], strict=False)
    assert len(unexpected_keys) == 0
    assert all([k.startswith('clip_model.') for k in missing_keys])
    print(f'Loading Residual Transformer {res_opt.name} from epoch {ckpt["ep"]}!')
    return res_transformer

In [20]:
res_opt_path = pjoin('checkpoints', opt.dataset_name, opt.res_name, 'opt.txt')
res_opt = get_opt(res_opt_path, device=opt.device)
res_model = load_res_model(res_opt,vq_opt)

Reading checkpoints\t2m\tres_nlayer8_ld384_ff1024_rvq6ns_cdp0.2_sw\opt.txt
latent_dim: 384, ff_size: 1024, nlayers: 8, nheads: 6, dropout: 0.2
Loading CLIP...


c:\Users\TP\Desktop\genAI\BAMM\venv\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Loading Residual Transformer tres_nlayer8_ld384_ff1024_rvq6ns_cdp0.2_sw from epoch 440!


C:\Users\TP\AppData\Local\Temp\ipykernel_20584\2724606168.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(pjoin(res_opt.checkpoints_dir, res_opt.datas

In [21]:
pjoin(res_opt.checkpoints_dir, res_opt.dataset_name, res_opt.name, 'model', 'net_best_fid.tar')

'./checkpoints\\t2m\\tres_nlayer8_ld384_ff1024_rvq6ns_cdp0.2_sw\\model\\net_best_fid.tar'

In [22]:
pjoin(model_opt.checkpoints_dir, model_opt.dataset_name, model_opt.name, 'model', 'latest.tar')

'./log/t2m\\t2m\\2024-02-14-14-27-29_8_GPT_officialTrans_2iterPrdictEnd\\model\\latest.tar'

In [23]:
assert res_opt.vq_name == model_opt.vq_name

In [24]:
t2m_transformer = load_trans_model(model_opt, opt, 'latest.tar')

##################################
#####Loading Length Predictor#####
##################################
length_estimator = load_len_estimator(model_opt)

latent_dim: 384, ff_size: 1024, nlayers: 8, nheads: 6, dropout: 0.2
Loading CLIP...


c:\Users\TP\Desktop\genAI\BAMM\gen_t2m_c.py:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(pjoin(model_opt.checkpoints_dir, model_opt.dataset_name, mode

Loading Transformer 2024-02-14-14-27-29_8_GPT_officialTrans_2iterPrdictEnd from epoch 1999!
Loading Length Estimator from epoch 11!


c:\Users\TP\Desktop\genAI\BAMM\gen_t2m_c.py:97: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(pjoin('checkpoints', opt.dataset_name, 'length_estimator', 'm

In [25]:
t2m_transformer.eval()
vq_model.eval()
res_model.eval()
length_estimator.eval()

res_model.to(opt.device)
t2m_transformer.to(opt.device)
vq_model.to(opt.device)
length_estimator.to(opt.device)

##### ---- Dataloader ---- #####
opt.nb_joints = 21 if opt.dataset_name == 'kit' else 22

mean = np.load(pjoin('checkpoints', opt.dataset_name, model_opt.vq_name, 'meta', 'mean.npy'))
std = np.load(pjoin('checkpoints', opt.dataset_name, model_opt.vq_name, 'meta', 'std.npy'))
def inv_transform(data):
    return data * std + mean

prompt_list = []
length_list = []

In [26]:
est_length = False

In [27]:
prompt_list = [
    "the person runs in forward direction",
     
]
est_length = True
length_list = []


In [28]:
if est_length:
    print("Since no motion length are specified, we will use estimated motion lengthes!!")
    text_embedding = t2m_transformer.encode_text(prompt_list)
    pred_dis = length_estimator(text_embedding)
    probs = F.softmax(pred_dis, dim=-1)  # (b, ntoken)
    token_lens = Categorical(probs).sample()  # (b, seqlen)
    # lengths = torch.multinomial()
else:
    token_lens = torch.LongTensor(length_list) // 4
    token_lens = token_lens.to(opt.device).long()

Since no motion length are specified, we will use estimated motion lengthes!!


In [ ]:
m_length = token_lens * 4
captions = prompt_list

sample = 0
kinematic_chain = t2m_kinematic_chain
converter = Joint2BVHConvertor()

In [ ]:
with torch.no_grad():
    mids, pred_len = t2m_transformer.generate(captions, token_lens,
                                    timesteps=opt.time_steps,
                                    cond_scale=opt.cond_scale,
                                    temperature=opt.temperature,
                                    topk_filter_thres=opt.topkr,
                                    gsample=opt.gumbel_sample,
                                    is_predict_len=opt.motion_length==-1)
    token_lens = pred_len
    m_length = token_lens*4
    # print(mids)
    # print(mids.shape)
    mids_1 = res_model.generate(mids, captions, token_lens, temperature=1, cond_scale=5)
    pred_motions = vq_model.forward_decoder(mids_1)
    pred_motions_1 = pred_motions.detach().cpu().numpy()
             
    data = inv_transform(pred_motions_1)

In [120]:
mids_1.shape

torch.Size([1, 50, 6])

In [73]:
quantized.shape

torch.Size([1, 116, 6])

In [ ]:
num_transition_token = 2
b = mids.shape[0]
half_token_length = (pred_len / 2).int()
idx_full_len = half_token_length >= 24
half_token_length[idx_full_len] -= 1

# tokens = -1 * torch.ones((b - 1, 50), dtype=torch.long)
tokens = -1 * torch.ones((b - 1, 50), dtype=torch.long, device=opt.device)

transition_train_length = []
for i in range(b - 1):
    i_index_motion = mids[i, :, 0]
    i1_index_motion = mids[i + 1, :, 0]

    left_end = half_token_length[i]
    right_start = left_end + num_transition_token
    end = right_start + half_token_length[i + 1]

    tokens[i, :left_end] = i_index_motion[pred_len[i] - left_end: pred_len[i]]
    tokens[i, left_end:right_start] = t2m_transformer.pad_id
    tokens[i, right_start:end] = i1_index_motion[:half_token_length[i + 1]]
    transition_train_length.append(end)

transition_train_length = torch.tensor(transition_train_length).to(mids.device).long()
tokens = tokens.scatter_(-1, transition_train_length[..., None], t2m_transformer.end_id).long()
inpainting_mask = tokens == t2m_transformer.pad_id
tokens[tokens == -1] = t2m_transformer.pad_id
inpaint_index = t2m_transformer.edit2(None, tokens)

all_tokens = []
for i in range(b - 1):
    all_tokens.append(mids[i, :pred_len[i]])
    inpaint_all = inpaint_index[i, inpainting_mask[i]].unsqueeze(-1)
    inpaint_all = torch.nn.functional.pad(inpaint_all, (0, 5), value=-1)
    all_tokens.append(inpaint_all)
all_tokens.append(mids[-1, :pred_len[-1]])
mids = torch.cat(all_tokens).unsqueeze(0)

Inbetween code here


In [ ]:
FPS = 20
t_sec=10 # addtion at this second
downsample_factor=5
frame_index = int(t_sec * FPS)
insert_frame = int(frame_index / downsample_factor)

In [122]:
insert_frame

40

In [109]:
mids_1[0][:pred_len[0]].shape

torch.Size([9, 6])

In [123]:


# Extract parts from quantized
quantized_base = quantized[0, :]  # Shape: [L, 6]
part1 = quantized_base[:insert_frame]        # Quantized before 10s
part2 = quantized_base[insert_frame:]        # Quantized after 10s


motion_A = part1[:, 0]        # Last tokens before 10s
motion_B = mids_1[0, : , 0]     # New motion to insert
motion_C = part2[:, 0]        # Tokens after 10s

# Prepare transition tokens
tokens = -1 * torch.ones((2, 50), dtype=torch.long, device=opt.device)
half_token_length = 24
num_transition_token = 2

# === Transition 1: part1 (A) to mids (B) ===
tokens[0, :half_token_length] = motion_A[-half_token_length:]
tokens[0, half_token_length:half_token_length + num_transition_token] = t2m_transformer.pad_id
tokens[0, half_token_length + num_transition_token:] = motion_B[:half_token_length]

# === Transition 2: mids (B) to part2 (C) ===
tokens[1, :half_token_length] = motion_B[-half_token_length:]
tokens[1, half_token_length:half_token_length + num_transition_token] = t2m_transformer.pad_id
tokens[1, half_token_length + num_transition_token:] = motion_C[:half_token_length]

# Predict transitions
tokens[tokens == -1] = t2m_transformer.pad_id
inpainting_mask = tokens == t2m_transformer.pad_id
inpaint_index = t2m_transformer.edit2(None, tokens)

# Extract transitions
transition1 = inpaint_index[0, inpainting_mask[0]].unsqueeze(-1)
transition1 = torch.nn.functional.pad(transition1, (0, 5), value=-1)

transition2 = inpaint_index[1, inpainting_mask[1]].unsqueeze(-1)
transition2 = torch.nn.functional.pad(transition2, (0, 5), value=-1)

# Construct final motion
final_motion = torch.cat([
    part1,         # original motion up to 10s
    transition1,   # transition: original → mids
    mids_1[0][:pred_len[0]],       # inserted new motion
    transition2,   # transition: mids → original continuation
    part2          # rest of original motion
], dim=0).unsqueeze(0)  # Add batch dim

# Final result
quantized = final_motion


In [78]:
pred_motions[0].shape

torch.Size([200, 263])

In [37]:
mids_1.shape

torch.Size([3, 50, 6])

In [132]:
quantized.shape

torch.Size([1, 313, 6])

In [129]:
pred_motions = vq_model.forward_decoder(mids_1)

In [130]:
pred_motions_2 =pred_motions.detach().cpu().numpy()

In [131]:
data2 = inv_transform(pred_motions_2)
joint2 = recover_from_ric(torch.from_numpy(data2).float(), 22).numpy()
_, ik_joint = converter.convert(joint2[0], filename="inframe1.bvh", iterations=100)

In [45]:
data.shape

(3, 200, 263)

In [124]:
trial1=vq_model.forward_decoder(quantized)

In [125]:
pred_motions_1 = trial1.detach().cpu().numpy()


In [126]:
data = inv_transform(pred_motions_1)

In [31]:
joint = recover_from_ric(torch.from_numpy(data).float(), 22).numpy()

In [32]:
joint.shape

(1, 200, 22, 3)

In [ ]:
_, ik_joint = converter.convert(joint[0], filename="inframe.bvh", iterations=100)

: 

In [183]:
joint_data.shape

(196, 263)

In [50]:
prompt_list = [
    'a figure sits on the chair.',
            #  'a person squatting raises both their arms above their head.',
             'a man jump forward.',
             'a person punches as if they are boxing.',
]

In [159]:
m_token_length = torch.ceil((pred_len)/4).int()
b=len(prompt_list)

In [160]:
half_token_length = (m_token_length/2).int()
idx_full_len = half_token_length >= 24
half_token_length[idx_full_len] = half_token_length[idx_full_len] - 1

mask_id = 512
tokens = -1*torch.ones((b-1, 50), dtype=torch.long)
transition_train_length = []

In [161]:
num_transition_token=2

In [162]:
for i in range(b-1):
    i_index_motion = mids[i]
    i1_index_motion = mids[i+1]

    left_end = half_token_length[i]
    right_start = left_end + num_transition_token
    end = right_start + half_token_length[i+1]

    tokens[i, :left_end] = i_index_motion[m_token_length[i]-left_end: m_token_length[i]]
    tokens[i, left_end:right_start] = mask_id
    tokens[i, right_start:end] = i1_index_motion[:half_token_length[i+1]]
    transition_train_length.append(end)

In [163]:
transition_train_length = torch.tensor(transition_train_length).to(mids.device)

In [164]:
ans=res_model.generate(tokens, captions[:-1], transition_train_length*4, temperature=1, cond_scale=5)

RuntimeError: index -1 is out of bounds for dimension 1 with size 513

In [108]:
import numpy as np
import os
import smplx
import json
import torch

In [114]:
from smplx import SMPL

model = SMPL(
    model_path='smplx/smpl',
    gender='NEUTRAL',   # or 'MALE' / 'FEMALE'
    batch_size=1
)

In [125]:
import json

with open("ballet/smpl/000000.json") as f:
    data = json.load(f)

# Assuming single person
person = data[0]
Rh = torch.tensor(person["Rh"], dtype=torch.float32)  # (1, 3)
Th = torch.tensor(person["Th"], dtype=torch.float32)  # (1, 3)
poses = torch.tensor(person["poses"], dtype=torch.float32).reshape(1, -1)  # (1, 72)
shapes = torch.tensor(person["shapes"], dtype=torch.float32)  # (1, 10)


In [132]:

person = data[0]
Rh = torch.tensor(person["Rh"], dtype=torch.float32)         # (1, 3)
Th = torch.tensor(person["Th"], dtype=torch.float32)         # (1, 3)
poses = torch.tensor(person["poses"], dtype=torch.float32)   # (1, 69)
shapes = torch.tensor(person["shapes"], dtype=torch.float32) # (1, 10)

# Use Rh as global_orient, and the rest as body_pose
output = model(
    global_orient=Rh,
    body_pose=poses,  # Skip the first 3 (which are 0s)
    betas=shapes,
    transl=Th
)

In [130]:
poses[0][:3]=Rh[0]

In [131]:
poses[0]

tensor([-2.8230, -0.1150,  0.2800, -0.1530, -0.0690, -0.0180,  0.1640,  0.1160,
        -0.0530,  0.2290,  0.0750, -0.1170,  0.8130, -0.1400,  0.1070,  0.1270,
        -0.0370,  0.0240, -0.1190,  0.1750,  0.1130, -0.0540, -0.1430,  0.0740,
        -0.0890,  0.0560,  0.0060, -0.0840,  0.1610,  0.0650,  0.0750,  0.0340,
        -0.0610,  0.0880,  0.1190, -0.0550, -0.0280, -0.1570, -0.3400,  0.0520,
         0.3480,  0.4080, -0.0250,  0.0070,  0.0170,  0.3600, -0.2150, -0.9730,
         0.3310,  0.2810,  0.9410,  0.3270, -0.8880,  0.1240,  0.3170,  1.0230,
        -0.0950,  0.0830, -0.1700, -0.2930, -0.1420,  0.1970,  0.2050, -0.2890,
        -0.1310, -0.3500, -0.0880,  0.1180,  0.3370])

tensor([[[ 0.1073, -1.1542,  0.6699],
         [ 0.1720, -1.0624,  0.6915],
         [ 0.0378, -1.0773,  0.7134],
         [ 0.1154, -1.2644,  0.6618],
         [ 0.1808, -0.6969,  0.6190],
         [-0.0348, -0.7124,  0.7923],
         [ 0.1308, -1.3818,  0.5992],
         [ 0.1200, -0.3121,  0.6868],
         [ 0.0946, -0.5091,  1.1063],
         [ 0.1306, -1.4104,  0.5485],
         [ 0.1467, -0.2639,  0.5659],
         [ 0.0312, -0.3927,  1.0935],
         [ 0.1405, -1.6162,  0.4825],
         [ 0.2162, -1.5260,  0.5150],
         [ 0.0583, -1.5381,  0.5287],
         [ 0.1529, -1.6397,  0.4047],
         [ 0.3105, -1.5145,  0.5057],
         [-0.0407, -1.5260,  0.5156],
         [ 0.3624, -1.2926,  0.6279],
         [-0.0922, -1.3041,  0.6240],
         [ 0.2776, -1.0754,  0.5484],
         [ 0.0235, -1.0999,  0.5309],
         [ 0.2325, -1.0038,  0.5523],
         [ 0.0812, -1.0393,  0.5189],
         [ 0.1569, -1.6355,  0.2892],
         [ 0.1250, -1.6862,  0.2994],
         [ 0

In [85]:
np.array(arr).shape

(2, 25, 3)

In [90]:
data=file[0]
import torch

In [115]:
poses=[i for i in reversed(data['poses'])]

In [116]:
poses

[[-0.466,
  -0.027,
  0.1,
  -0.241,
  -0.064,
  -0.027,
  0.179,
  0.09,
  -0.057,
  0.233,
  0.059,
  -0.115,
  0.907,
  -0.153,
  0.129,
  0.114,
  -0.046,
  0.02,
  -0.112,
  0.176,
  0.113,
  -0.048,
  -0.135,
  0.075,
  -0.091,
  0.043,
  0.006,
  -0.092,
  0.169,
  0.056,
  0.06,
  0.021,
  -0.079,
  0.086,
  0.126,
  -0.083,
  -0.03,
  -0.21,
  -0.359,
  0.041,
  0.343,
  0.408,
  -0.028,
  0.034,
  0.028,
  0.352,
  -0.247,
  -0.977,
  0.322,
  0.246,
  0.959,
  0.328,
  -0.831,
  0.123,
  0.299,
  0.987,
  -0.108,
  0.102,
  -0.161,
  -0.285,
  -0.133,
  0.187,
  0.208,
  -0.286,
  -0.132,
  -0.347,
  -0.092,
  0.121,
  0.357]]

In [134]:
arr=np.array([output.joints.numpy()[0],output.joints.numpy()[0]])

In [135]:
arr=np.array(arr)

In [95]:
joints=[output.joints[0],output.joints[0]]

C:\Users\TP\AppData\Local\Temp\ipykernel_7456\2736449939.py:1: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  np.array(joints).shape
C:\Users\TP\AppData\Local\Temp\ipykernel_7456\2736449939.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(joints).shape


(2,)

In [ ]:
from visualization.joints2bvh import Joint2BVHConvertor
converter = Joint2BVHConvertor()

In [28]:
temp=np.array(arr)

In [72]:
temp=np.load('joints.npy')

In [101]:
openpose_to_smpl_idx = [
    8,   # Pelvis (MidHip)
    12,  # L_Hip
    9,   # R_Hip
    1,   # Spine1 (Neck as proxy)
    13,  # L_Knee
    10,  # R_Knee
    0,   # Spine2 (Nose as proxy)
    14,  # L_Ankle
    11,  # R_Ankle
    1,   # Spine3 (Neck)
    19,  # L_Foot (LBigToe)
    22,  # R_Foot (RBigToe)
    1,   # Neck
    5,   # L_Collar
    2,   # R_Collar
    0,   # Head (Nose)
    5,   # L_Shoulder
    2,   # R_Shoulder
    6,   # L_Elbow
    3,   # R_Elbow
    7,   # L_Wrist
    4,   # R_Wrist
    7,   # L_Hand (approx: same as wrist)
    4,   # R_Hand (approx: same as wrist)
]


In [102]:
arr=np.array(arr)

In [103]:
smpl_joints = arr[:, openpose_to_smpl_idx, :]  # shape: (N, 24, 3)


In [105]:
# Flip Z to fix orientation
smpl_joints[..., 2] *= -1

# Optional: normalize position around pelvis
smpl_joints -= smpl_joints[:, 0:1, :]


In [73]:
temp.shape

(1808, 127, 3)

In [100]:
import json
arr=[]
for i in os.listdir('jazz/keypoints3d')[:500]:
    with open(os.path.join('jazz/keypoints3d',i),'r') as f:

        file=json.load(f)
        joints=file[0]['keypoints3d']
        arr.append(joints)




In [81]:
with open(os.path.join('ballet/smpl','000000.json'),'r') as f:

        file=json.load(f)

In [82]:
np.array(file[0])

array({'id': 0, 'Rh': [[-2.823, -0.115, 0.28]], 'Th': [[0.109, -0.933, 0.642]], 'poses': [[-0.49, -0.036, 0.086, -0.153, -0.069, -0.018, 0.164, 0.116, -0.053, 0.229, 0.075, -0.117, 0.813, -0.14, 0.107, 0.127, -0.037, 0.024, -0.119, 0.175, 0.113, -0.054, -0.143, 0.074, -0.089, 0.056, 0.006, -0.084, 0.161, 0.065, 0.075, 0.034, -0.061, 0.088, 0.119, -0.055, -0.028, -0.157, -0.34, 0.052, 0.348, 0.408, -0.025, 0.007, 0.017, 0.36, -0.215, -0.973, 0.331, 0.281, 0.941, 0.327, -0.888, 0.124, 0.317, 1.023, -0.095, 0.083, -0.17, -0.293, -0.142, 0.197, 0.205, -0.289, -0.131, -0.35, -0.088, 0.118, 0.337]], 'shapes': [[-0.177, -0.016, -0.004, 0.039, -0.003, 0.003, -0.001, 0.003, 0.0, -0.002]]},
      dtype=object)

In [68]:
temp=np.load('ballet.npy')

In [69]:
temp.shape

(8372, 127, 3)

In [69]:
arr[0].shape

(45, 3)

In [30]:

from visualization.joints2bvh import Joint2BVHConvertor
converter = Joint2BVHConvertor()
# new_bvh=converter.convert(arr, "ballet.bvh", iterations=100, fps=30)

In [6]:
_, ik_joint = converter.convert(temp, "try_new.bvh", iterations=100, fps=28)

In [8]:
n = temp["poses"].shape[0]

In [9]:
pose = torch.from_numpy(temp["poses"][:n]).to(torch.float32)

In [10]:
pose.shape

torch.Size([1808, 165])

In [14]:
from fastapi import FastAPI, Query, HTTPException
from fastapi.responses import FileResponse
import subprocess
import os
import glob
import sys
import shutil
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from gen_t2m import BVHGenerator
from fastapi import Body

ImportError: cannot import name 'BVHGenerator' from 'gen_t2m' (c:\Users\TP\Desktop\genAI\BAMM\gen_t2m.py)

In [16]:
from gen_t2m import BVHGenerator

ImportError: cannot import name 'BVHGenerator' from 'gen_t2m' (c:\Users\TP\Desktop\genAI\BAMM\gen_t2m.py)

In [10]:
res_name: str = "tres_nlayer8_ld384_ff1024_rvq6ns_cdp0.2_sw"
name: str = "2024-02-14-14-27-29_8_GPT_officialTrans_2iterPrdictEnd"
text_prompt: str=""
motion_length: int = -1
repeat_times: int = 1
gpu_id: int = -1
seed: int = 1
ext: str = "generation_name_nopredlen"

command = [
         sys.executable, "gen_t2m.py",
        "--res_name", res_name,
        "--name", name,
        "--text_prompt", text_prompt,
        "--motion_length", str(motion_length),
        "--repeat_times", str(repeat_times),
        "--gpu_id", str(gpu_id),
        "--seed", str(seed),
        "--ext", ext
    ]
print(command)
subprocess.run(command, capture_output=True, text=True)

['c:\\Users\\TP\\Desktop\\genAI\\BAMM\\venv\\Scripts\\python.exe', 'gen_t2m.py', '--res_name', 'tres_nlayer8_ld384_ff1024_rvq6ns_cdp0.2_sw', '--name', '2024-02-14-14-27-29_8_GPT_officialTrans_2iterPrdictEnd', '--text_prompt', '', '--motion_length', '-1', '--repeat_times', '1', '--gpu_id', '-1', '--seed', '1', '--ext', 'generation_name_nopredlen']


CompletedProcess(args=['c:\\Users\\TP\\Desktop\\genAI\\BAMM\\venv\\Scripts\\python.exe', 'gen_t2m.py', '--res_name', 'tres_nlayer8_ld384_ff1024_rvq6ns_cdp0.2_sw', '--name', '2024-02-14-14-27-29_8_GPT_officialTrans_2iterPrdictEnd', '--text_prompt', '', '--motion_length', '-1', '--repeat_times', '1', '--gpu_id', '-1', '--seed', '1', '--ext', 'generation_name_nopredlen'], returncode=0, stdout='------------ Options -------------\nbatch_size: 32\ncheckpoints_dir: ./log/t2m\ncond_scale: 4\ndataset_name: t2m\ndropout: 0.2\next: generation_name_nopredlen\nff_size: 1024\nforce_mask: False\ngpu_id: -1\ngumbel_sample: False\nis_train: False\nlatent_dim: 384\nmask_edit_section: None\nmax_motion_length: 196\nmotion_length: -1\nn_heads: 6\nn_layers: 8\nname: 2024-02-14-14-27-29_8_GPT_officialTrans_2iterPrdictEnd\nnum_batch: 2\nrepeat_times: 1\nres_name: tres_nlayer8_ld384_ff1024_rvq6ns_cdp0.2_sw\nseed: 1\nsource_motion: example_data/000612.npy\ntemperature: 1.0\ntext_path: \ntext_prompt: \ntime_step

In [ ]:
try=BVHGenerator()

In [12]:
text_prompt="|".join(test_prompt)

In [8]:
res=create_bvh_from_in(text_prompt)

NameError: name 'opt' is not defined

In [1]:
import smplx
import numpy as np
import torch
import json
import os


In [ ]:
from smplx import create





In [13]:
from smplx import create
import torch
import json

model_path = './smplx'  # path to folder containing SMPL_NEUTRAL.pkl
model = create(model_path, model_type='smpl', gender='neutral')

# Load a sample file
with open('ballet/smpl/000000.json') as f:
    smpl_data = json.load(f)[0]

#
# Convert to tensors
global_orient = torch.tensor(smpl_data['Rh'], dtype=torch.float32)     # (1, 3)
body_pose     = torch.tensor(smpl_data['poses'], dtype=torch.float32)  # (1, 69)
betas         = torch.tensor(smpl_data['shapes'], dtype=torch.float32) # (1, 10)
transl        = torch.tensor(smpl_data['Th'], dtype=torch.float32)     # (1, 3)

# Run SMPL
output = model(
    global_orient=global_orient,
    body_pose=body_pose,
    betas=betas,
    transl=transl,
    return_verts=True
)

verts = output.vertices  # (1, 6890, 3)

torch.Size([1, 45, 3])

In [1]:
import json
import numpy as np
import torch
import smplx
import trimesh
from pythreejs import *

In [5]:
with open("ballet/smpl_joints/000000.json", "r") as f:
    keypoints = np.array(json.load(f)[0]['joints3d'])


In [6]:
model_path = "smplx"  # CHANGE THIS to your actual path
model = smplx.create(model_path, model_type='smpl', gender='neutral', use_pca=False)

(45, 3)

In [15]:
output = model(betas=torch.zeros(1, 10),
               body_pose=torch.zeros(1, 69),
               global_orient=torch.zeros(1, 3),
               transl=torch.zeros(1, 3))
vertices = output.vertices.detach().cpu().numpy().squeeze()
faces = model.faces

In [9]:
smpl_geom = BufferGeometry(
    attributes={
        "position": BufferAttribute(vertices.astype(np.float32)),
        "index": BufferAttribute(faces.astype(np.uint32).flatten())
    }
)

In [10]:
smpl_material = MeshStandardMaterial(color='peachpuff', metalness=0.1, roughness=0.6, side='DoubleSide')
smpl_mesh = Mesh(geometry=smpl_geom, material=smpl_material)

In [11]:
joint_spheres = []
for joint in keypoints:
    sphere_geom = SphereGeometry(radius=0.02)
    sphere_material = MeshStandardMaterial(color='skyblue')
    sphere = Mesh(geometry=sphere_geom, material=sphere_material, position=tuple(joint))
    joint_spheres.append(sphere)

In [12]:
scene = Scene(children=[smpl_mesh, *joint_spheres, AmbientLight(intensity=0.5)])

# Add camera
camera = PerspectiveCamera(position=[2, 0, 2], fov=50,
                           children=[DirectionalLight(position=[3, 5, 1], intensity=0.5)])

# Renderer
renderer = Renderer(camera=camera, scene=scene, controls=[OrbitControls(controlling=camera)],
                    width=800, height=600)

display(renderer)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(intensity=0.5, position=(3.0, 5.0, 1.0), quaterni…

In [350]:
with open("ballet/smpl/000000.json", "r") as f:
    data = json.load(f)[0]

In [351]:
with open("ballet/smpl/000000.json", "r") as f:
    data = json.load(f)[0]
pose = torch.tensor(data['poses'], dtype=torch.float32)
betas = torch.tensor(data['shapes'], dtype=torch.float32)
Rh = torch.tensor(data['Rh'], dtype=torch.float32)
# Rh=torch.zeros((1,3))
Th = torch.tensor(data['Th'], dtype=torch.float32)


In [ ]:
# Rh = Rh * torch.tensor([1, math.pi, 1])

In [342]:
Rh

tensor([[-2.8230, -0.3613,  0.2800]])

In [352]:
model_path = "smplx"  # CHANGE this to your actual SMPL model path
model = smplx.create(model_path, model_type='smpl', gender='neutral', use_pca=False)

In [419]:
output = model(betas=betas, body_pose=pose, global_orient=Rh, transl=Th)
vertices = output.vertices.detach().cpu().numpy().squeeze()
faces = model.faces

In [420]:
joints=output.joints

In [346]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt


In [483]:
#  Convert to BufferGeometry
geometry = BufferGeometry(
    attributes={
        'position': BufferAttribute(vertices.astype(np.float32)),
        'index': BufferAttribute(faces.astype(np.uint32).flatten())
    }
)
material = MeshStandardMaterial(
    color='peachpuff',
    roughness=0.6,
    metalness=0.1,
    side='DoubleSide',
    transparent=True,
    opacity=0.3  # Adjust to control transparency level
)
mesh = Mesh(geometry=geometry, material=material)
# mesh.scale = [1, -1, 1]

# Scene setup
scene = Scene(children=[
    mesh,
    AmbientLight(intensity=0.5),
    DirectionalLight(position=[3, 5, 1], intensity=0.6)
])

camera = PerspectiveCamera(position=[5, 3, 1.5], fov=45)
controls = OrbitControls(controlling=camera)

renderer = Renderer(camera=camera, scene=scene, controls=[controls],
                    width=800, height=600)
display(renderer)

Renderer(camera=PerspectiveCamera(fov=45.0, position=(5.0, 3.0, 1.5), projectionMatrix=(1.0, 0.0, 0.0, 0.0, 0.…

In [475]:
def smpl_to_threejs(coords):
    coords = np.array(coords)
    coords[..., 1] *= -1  # Flip Y
    coords[..., 2] *= -1  # Flip Z ✅
    # coords[..., 0] *= -1  # Flip Z
    return coords


In [476]:
joints=output.joints

In [477]:
j=smpl_to_threejs(joints)

In [484]:
# with open("ballet/smpl_joints/000000.json", "r") as f:
#     joints = np.array(json.load(f)[0]['joints3d'])

# Create joint spheres
joint_spheres = []
for joint in j[0]:
    sphere_geom = SphereGeometry(radius=0.01)  # Adjust radius if needed
    sphere_material = MeshStandardMaterial(color='skyblue')
    sphere = Mesh(geometry=sphere_geom, material=sphere_material, position=tuple(joint))
    joint_spheres.append(sphere)

# Add everything to the scene again
scene = Scene(children=[
    mesh,  # your SMPL mesh
    *joint_spheres,  # the spheres for joints
    AmbientLight(intensity=0.5),
    DirectionalLight(position=[3, 5, 1], intensity=0.6)
])

camera = PerspectiveCamera(position=[5, 3, 1.5], fov=45)
controls = OrbitControls(controlling=camera)

renderer = Renderer(camera=camera, scene=scene, controls=[controls],
                    width=800, height=600)

display(renderer)

Renderer(camera=PerspectiveCamera(fov=45.0, position=(5.0, 3.0, 1.5), projectionMatrix=(1.0, 0.0, 0.0, 0.0, 0.…

In [396]:
np.array(joints).shape

(1, 45, 3)

In [395]:
j.shape

(1, 45, 3)

In [398]:

from visualization.joints2bvh import Joint2BVHConvertor
converter = Joint2BVHConvertor()


In [402]:
new_bvh=converter.convert(np.array(joints), "ballet.bvh", iterations=100, fps=30)

c:\Users\TP\Desktop\genAI\BAMM\venv\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\TP\Desktop\genAI\BAMM\venv\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [508]:
y,z


NameError: name 'y' is not defined

In [13]:
from scipy.spatial.transform import Rotation as R

def rotate_scene(points, axis='x', degrees=15):
    rot = R.from_euler(axis, degrees, degrees=True).as_matrix()
    return np.matmul(points, rot.T)


In [2]:
def smpl_to_threejs(coords):
    coords = np.array(coords)

    # Flip axes to convert SMPL to Three.js-friendly coordinate space
    coords[..., 1] *= -1  # Flip Y
    coords[..., 2] *= -1  # Flip Z

    # Apply tilt correction (rotate 15 degrees arou nd X-axis to straighten body)
    # coords = rotate_scene(coords, axis='x', degrees=15)

    return coords


In [ ]:
import os
import json
import torch
import numpy as np
import smplx

# SMPL model setup
model_path = "smplx"  # CHANGE this to your actual SMPL model path
model = smplx.create(model_path, model_type='smpl', gender='neutral', use_pca=False)

joints_all = []

# Path to your directory
json_dir = "jazz/smpl"
Rh_all=[]
poses_all=[]

# Loop through all .json files in the directory
for file_name in sorted(os.listdir(json_dir)[:3000]):
    if file_name.endswith(".json"):
        with open(os.path.join(json_dir, file_name), "r") as f:
            data = json.load(f)[0]

        # Load pose params
        pose = torch.tensor(data['poses'], dtype=torch.float32)
        betas = torch.tensor(data['shapes'], dtype=torch.float32)
        Rh = torch.tensor(data['Rh'],dtype=torch.float32)

        Th = torch.tensor(data['Th'], dtype=torch.float32)

        
        output = model(
            betas=betas,
            body_pose=pose,
            global_orient=Rh,
            transl=Th
        )

        # Rh=Rh*torch.tensor([1,-1,-1])


        # Forward pass
        output = model(betas=betas, body_pose=pose, global_orient=Rh, transl=Th)
        joints = output.joints.detach().cpu().numpy().squeeze()
        joints=smpl_to_threejs(joints)
        Rh_all.append(Rh)
        poses_all.append(torch.cat([Rh[0], pose[0]], dim=0))
        joints_all.append(joints)

# Convert to a NumPy array if needed
joints_all = np.array(joints_all)  # shape: (num_frames, num_joints, 3)
Rh_all = np.array(Rh_all)
print("Collected joints shape:", joints_all.shape)


Collected joints shape: (3000, 45, 3)


C:\Users\TP\AppData\Local\Temp\ipykernel_15352\1919913801.py:52: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  Rh_all = np.array(Rh_all)
C:\Users\TP\AppData\Local\Temp\ipykernel_15352\1919913801.py:52: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Rh_all = np.array(Rh_all)


In [11]:
# Convert list of 72D pose vectors to tensor
poses_all_tensor = torch.stack(poses_all) 
poses_all_tensor = poses_all_tensor.view(-1, 24, 3)


In [14]:
positions_np = poses_all_tensor.detach().cpu().numpy() 

In [16]:

from visualization.joints2bvh import Joint2BVHConvertor
converter = Joint2BVHConvertor()
new_bvh=converter.convert(positions_np, "jazz.bvh", iterations=100,fps=50)

In [17]:
import shutil

src = "jazz.bvh"
path = r"D:\Tanmay Files\RA\threejs"

shutil.copy(src, path)  # Copies the file


'D:\\Tanmay Files\\RA\\threejs\\jazz.bvh'

In [55]:
import shutil

src = "ballet1.bvh"
path = r"D:\Tanmay Files\RA\threejs"

shutil.copy(src, path)  # Copies the file


'D:\\Tanmay Files\\RA\\threejs\\ballet1.bvh'

In [81]:
import shutil

src = "ballet2.bvh"
path = r"D:\Tanmay Files\RA\threejs"

shutil.copy(src, path)  # Copies the file


'D:\\Tanmay Files\\RA\\threejs\\ballet2.bvh'

In [25]:
from scipy.spatial.transform import Rotation as R


In [39]:
import os
import json
import torch
import numpy as np
import smplx

# SMPL model setup
model_path = "smplx"  # CHANGE this to your actual SMPL model path
model = smplx.create(model_path, model_type='smpl', gender='neutral', use_pca=False)

joints_all = []

# Path to your directory
json_dir = "graham/smpl"
all_frames_rotations = []
all_th=[]
# Loop through all .json files in the directory
for file_name in sorted(os.listdir(json_dir)):
    if file_name.endswith(".json"):
        with open(os.path.join(json_dir, file_name), "r") as f:
            data = json.load(f)[0]

        # Load pose params
        pose = torch.tensor(data['poses'], dtype=torch.float32)
        betas = torch.tensor(data['shapes'], dtype=torch.float32)
        Rh = torch.tensor(data['Rh'],dtype=torch.float32)

        Th = torch.tensor(data['Th'], dtype=torch.float32)

        
        

        # Rh=Rh*torch.tensor([1,-1,-1])
        

        # Forward pass
        full_pose = torch.cat([Rh, pose], dim=1)  # (1, 72)

    # Convert axis-angle to rotation matrices (24 joints × 3D)
        rotvecs = full_pose.view(-1, 3).numpy()  # (24, 3)
        rot_mats = R.from_rotvec(rotvecs).as_matrix()  # (24, 3, 3)

        all_frames_rotations.append(rot_mats.tolist())  # Save as JSON-serializable
        all_th.append(Th.tolist())

# Save all rotation matrices for all frames
with open("all_rotations.json", "w") as f:
    json.dump(all_frames_rotations, f)

with open("all_th.json", "w") as f:
    json.dump(all_th, f)

(8372, 1, 3)

In [40]:
import shutil

src_path="all_rotations.json"
path = r"D:\Tanmay Files\RA\threejs\graham"
shutil.copy(src=src_path,dst=path)

'D:\\Tanmay Files\\RA\\threejs\\graham\\all_rotations.json'

In [41]:
import shutil

src_path="all_th.json"
path = r"D:\Tanmay Files\RA\threejs\graham"
shutil.copy(src=src_path,dst=path)

'D:\\Tanmay Files\\RA\\threejs\\graham\\all_th.json'